In [25]:
import pandas as pd
from thefuzz import process

In [19]:
archivos = ['MercaWeek_Carulla.xlsx', 'MercaWeek_Exito.xlsx'
            , 'Cumpleaños_Carulla.xlsx']


for archivo in archivos:
    df = pd.read_excel(archivo)
    
    if 'Precio regular' in df.columns and 'Precio promocional' in df.columns:

        df = df[df['Precio regular'] != df['Precio promocional']]
        
        df['% de descuento'] = round((1 - df['Precio promocional'] / df['Precio regular']) * 100)
        df['% de descuento'] = df['% de descuento'].astype(int).astype(str) + '%'
        

        df.to_excel(archivo, index=False)
        print(f'Archivo {archivo} actualizado correctamente con la columna % de descuento.')
    else:
        print(f'El archivo {archivo} no contiene las columnas necesarias.')
    

Archivo MercaWeek_Carulla.xlsx actualizado correctamente con la columna % de descuento.
Archivo MercaWeek_Exito.xlsx actualizado correctamente con la columna % de descuento.
Archivo Cumpleaños_Carulla.xlsx actualizado correctamente con la columna % de descuento.


In [38]:
df_comparables = pd.read_excel('comparables.xlsx')
df_comparables.head()

,LLAVE,EAN,Cod. Article,Cod. Article.1,Gramaje,UM,Artículos encontrados comparables Éxito Suba,LINK
0,27UN,7.702511e+12,27,ARROZ DIANA BOLSA 500g,500.0,UN,Arroz Blanco Diana X 500 Gr DIANA 500 gr,https://www.exito.com/arroz-diana-500-gr-479512/p
1,27R01,2.000000e+12,27,ARROZ DIANA BOLSA 12500g,12500.0,R01,Arroz Blanco Diana X 500 Gr DIANA 12500 gr,https://www.exito.com/arroz-blanco-arroba-1250...
2,34UN,7.702084e+12,34,HARINA PAN MAIZ BLANCO BOLSA 1000g,1000.0,UN,harina-pan-maiz-blanca-x-1000-gr-,https://www.exito.com/harina-pan-maiz-blanca-x...
3,38UN,7.702032e+12,38,CAFE SELLO ROJO BOLSA 250g,250.0,UN,CAFE MOLIDO SELLO ROJO 250 gr,https://www.exito.com/cafe-molido-34895/p
4,43UN,7.703812e+12,43,SAL REFISAL ALTA PUREZA BOLSA 1000 g,1000.0,UN,sal-refinada-1000-gr-,https://www.exito.com/sal-refinada-1000-gr-345...


In [52]:
df_comparables['Mejor_Articulo_Exito'] = ''
df_comparables['Puntaje'] = 0
df_comparables['Comparacion'] = ''

umbral = 87

for archivo in archivos:
    df = pd.read_excel(archivo)
    for i, row in df_comparables.iterrows():
        articulo_ara = row['LINK']
        mejor_match = process.extractOne(articulo_ara, df['Link'])
    
        if mejor_match[1] >= umbral:
            df_comparables.at[i, 'Mejor_Articulo_Exito'] = mejor_match[0]
            df_comparables.at[i, 'Puntaje'] = mejor_match[1]
            df_comparables.at[i, 'Comparacion'] = 'Comparable'
        else:
            df_comparables.at[i, 'Mejor_Articulo_Exito'] = 'No comparable'
            df_comparables.at[i, 'Puntaje'] = mejor_match[1]
            df_comparables.at[i, 'Comparacion'] = 'No comparable'
    
    df_comparables.to_excel(archivo[:-5] + 'Comparables.xlsx', index=False)
# Guardar los resultados en un nuevo archivo Excel
#df_comparables.to_excel('resultado_matches.xlsx', index=False)
#df_comparables.head()